In [ ]:
import py3Dmol
import ipywidgets as widgets
from IPython.display import display, clear_output
from tkinter import Tk, filedialog
from Bio import PDB
import shutil

# Global viewer variable to maintain state
viewer = None
modified_pdb_path = "modified_structure.pdb"

def load_pdb_file():
    root = Tk()
    root.withdraw()
    pdb_path = filedialog.askopenfilename(title="Select PDB File", filetypes=[("PDB files", "*.pdb")])
    if pdb_path:
        with open(pdb_path, "r") as file:
            pdb_data = file.read()
        shutil.copy(pdb_path, modified_pdb_path)  # Create a modifiable copy
        return pdb_data, pdb_path
    return None, None

def modify_pdb(residue_number, modification, remove=False, dehydrate=False):
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("protein", modified_pdb_path)
    io = PDB.PDBIO()

    ptm_atoms = {
        "Phosphorylation": ["P", "O1", "O2", "O3"],  
        "Acetylation": ["C1", "O1", "CH3"],  
        "Methylation": ["C1", "H1", "H2", "H3"]  
    }
    dehydration_atoms = ["HOH", "O"]  # Water molecules to remove

    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.id[1] == residue_number:
                    if dehydrate:
                        # Remove water molecules from the selected residue
                        for atom_name in dehydration_atoms:
                            if atom_name in residue:
                                residue.detach_child(atom_name)
                    elif remove:
                        # Remove PTM atoms if they exist
                        for atom_name in ptm_atoms.get(modification, []):
                            if atom_name in residue:
                                residue.detach_child(atom_name)
                    else:
                        # Add PTM atoms
                        attachment_candidates = {
                            "Phosphorylation": ["OG", "OH", "O"],
                            "Acetylation": ["N"],
                            "Methylation": ["CE", "C"]
                        }

                        attachment_atom = None
                        for atom_name in attachment_candidates.get(modification, []):
                            if atom_name in residue:
                                attachment_atom = residue[atom_name]
                                break

                        if not attachment_atom:
                            print(f"⚠️ Warning: No valid attachment atom found in residue {residue_number}. Skipping PTM.")
                            return

                        ref_coord = attachment_atom.coord
                        offsets = [(1.5, 0.0, 0.0), (2.0, 0.5, 0.5), (2.0, -0.5, -0.5), (2.5, 0.0, 0.0)]

                        for atom_name, offset in zip(ptm_atoms[modification], offsets):
                            new_coord = (ref_coord[0] + offset[0], ref_coord[1] + offset[1], ref_coord[2] + offset[2])
                            new_atom = PDB.Atom.Atom(atom_name, new_coord, 1.0, 0.0, " ", atom_name, 0, "C")
                            residue.add(new_atom)

    io.set_structure(structure)
    io.save(modified_pdb_path)

def save_modified_pdb():
    root = Tk()
    root.withdraw()
    save_path = filedialog.asksaveasfilename(defaultextension=".pdb", filetypes=[("PDB files", "*.pdb")])
    if save_path:
        shutil.copy(modified_pdb_path, save_path)
        print(f"File saved successfully at: {save_path}")

def visualize_pdb_with_selection(pdb_data):
    global viewer
    output = widgets.Output()
    
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("protein", modified_pdb_path)
    available_residues = [residue.id[1] for model in structure for chain in model for residue in chain]
    print("Available Residues:", available_residues)
    
    def update_viewer(residue_number, modification, remove=False, dehydrate=False):
        global viewer
        with output:
            clear_output(wait=True)
            print(f"Selected Residue: {residue_number}")
            
            if residue_number not in available_residues:
                print(f"Residue {residue_number} not found in PDB structure.")
                return
            
            if dehydrate:
                display(widgets.Label(f"Dehydrating Residue {residue_number}..."))
                modify_pdb(residue_number, modification, dehydrate=True)
            elif remove:
                display(widgets.Label(f"Removing PTM from Residue {residue_number}..."))
                modify_pdb(residue_number, modification, remove=True)
            else:
                display(widgets.Label(f"Applying {modification} to Residue {residue_number}..."))
                modify_pdb(residue_number, modification)
            
            if viewer:
                viewer.removeAllModels()

            viewer = py3Dmol.view(width=800, height=600)
            with open(modified_pdb_path, "r") as file:
                modified_pdb_data = file.read()
            viewer.addModel(modified_pdb_data, "pdb")
            
            viewer.setStyle({"cartoon": {"color": "spectrum"}})
            viewer.addStyle({"resi": str(residue_number)}, {"stick": {"color": "red"}})
            viewer.zoomTo()
            display(viewer.show())
    
    residue_selector = widgets.IntText(value=1, description='Residue:', continuous_update=False)
    ptm_selector = widgets.Dropdown(
        options=["None", "Phosphorylation", "Acetylation", "Methylation"],
        value="None",
        description='PTM:'
    )
    apply_button = widgets.Button(description="Apply Modification")
    remove_button = widgets.Button(description="Remove Modification")
    dehydrate_button = widgets.Button(description="Dehydrate Residue")
    save_button = widgets.Button(description="Save PDB File")
    
    save_button.on_click(lambda b: save_modified_pdb())
    apply_button.on_click(lambda b: update_viewer(residue_selector.value, ptm_selector.value, remove=False))
    remove_button.on_click(lambda b: update_viewer(residue_selector.value, ptm_selector.value, remove=True))
    dehydrate_button.on_click(lambda b: update_viewer(residue_selector.value, ptm_selector.value, dehydrate=True))
    
    display(residue_selector, ptm_selector, apply_button, remove_button, dehydrate_button, save_button, output)
    
    with output:
        clear_output(wait=True)
        print("Select a residue and PTM, then click 'Apply Modification', 'Remove Modification', or 'Dehydrate Residue' to update the structure.")

pdb_data, pdb_path = load_pdb_file()
if pdb_data:
    visualize_pdb_with_selection(pdb_data)
else:
    print("No PDB file selected.")




Available Residues: [1, 2, 3, 4, 5, 6, 7, 8, 9]


IntText(value=1, description='Residue:')

Dropdown(description='PTM:', options=('None', 'Phosphorylation', 'Acetylation', 'Methylation'), value='None')

Button(description='Apply Modification', style=ButtonStyle())

Button(description='Remove Modification', style=ButtonStyle())

Button(description='Dehydrate Residue', style=ButtonStyle())

Button(description='Save PDB File', style=ButtonStyle())

Output()